In [0]:
%load_ext autoreload
%autoreload 2

In [2]:
# for Google Colab only: Mount my Google Drive
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [3]:
pwd

'/gdrive'

In [4]:
# for Google Colab only: Navigate to my snaug project folder
cd 'My Drive/Colab Notebooks/snaug'

/gdrive/My Drive/Colab Notebooks/snaug


In [5]:
# peek at current system Path settings
import sys
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

In [6]:
# let's add my project path so that I can load my custom written
# library functions saved in '/lib' 
sys.path.append("/gdrive/My Drive/Colab Notebooks/snaug")
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/gdrive/My Drive/Colab Notebooks/snaug']

In [7]:
#
# Load trained models to generate new text
#

import string
import textwrap
import pickle
import sys
from random import randint

from keras.models import load_model

from lib.nlplstm_class import (TFModelLSTMCharToken, TFModelLSTMWordToken, 
                               TFModelLSTMWord2vec) 
from lib.data_common import (load_doc, save_doc, clean_doc, prepare_char_tokens,
                             build_token_lines, prepare_text_tokens, load_word2vec,
                             sample_predict, generate_seq_of_chars, generate_seq_of_words)
from lib.data_common import pathfinder_textfile, fixed_length_token_textfile

Using TensorFlow backend.


In [0]:
# parameters for character tokenisation model
maxlen = 40
step = 3

In [0]:
# load fixed-length lines of tokens
doc = load_doc(fixed_length_token_textfile)
lines = doc.split('\n')

In [0]:
# generate word_seed_text for word tokenisation models
# generate separate char_seed_text for char tokenisation model, using
# maxlen rightmost characters of word seed text
word_seed_text = lines[randint(0,len(lines))]
char_seed_text = word_seed_text[-maxlen:]

In [0]:
# LSTM model that uses character tokenisation  
#
# load document
text = load_doc(pathfinder_textfile).lower()

In [67]:
# tokenize character data and separate into features and target for LSTM training
X, y, char2indices, indices2char, num_unique_char = prepare_char_tokens(text, maxlen, step)

corpus length: 76633
total chars: 57
number of sequences: 25531
number of next_chars: 25531


In [68]:
# load trained model
textgen_model_1 = load_model('./model/pathfinder_chartoken_model_50_epoch_withcuda_model.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [69]:
# select seed text
#start_index = randint(0, len(text) - maxlen - 1)
#seed_text = text[start_index: start_index + maxlen]
print()
print('> using character tokenisation')
print('> seed text:')
print(textwrap.fill('%s' % (char_seed_text), 80) + '\n')


> using character tokenisation
> seed text:
ble with a successful dc knowledge local



In [0]:
# generate new text
#temperature_table = [0.2, 0.7,1.2]
#temperature_table = [0.07, 0.1, 0.5, 0.7] 
temperature_table = [0, 0.7] 

In [71]:
for temperature in temperature_table:
    generated = generate_seq_of_chars(textgen_model_1, num_unique_char, char2indices, 
                    indices2char, char_seed_text, maxlen, 300, temperature)    
    print(">> generated text (temperature: {})".format(temperature))
    print(textwrap.fill('%s' % (generated), 80))
    print()

>> generated text (temperature: 0)
 comes a dissong creck abread to eaper the ring to hear the sandpoint in the
wind a small captain now a points and styyengess-demer, and a scccentions from a
for the rap, the beliening of shobthess gropp and pcs who elemental in surprised
to hel make a for gite and stealsh with a worken of golds wit

>> generated text (temperature: 0.7)
 combat—yagg, and is she robb as magnimar’s hork samp, and as not a points and
following the beat of gold, in simpating the mapical mumber she wastreaks he
enter the pcs may of sandpoint’s strypeled betore and to searing the maps nom a
can grack fagilies she remares staight acamem, and for sceeters



In [0]:
# 
# Loading text data that uses word tokenisation
#

In [0]:
# tokenize and separate into features and target
X, y, seq_length, vocab_size, tokenizer = prepare_text_tokens(lines)

In [0]:
#
# Word tokenization with word embedding model
#

In [105]:
# load trained model
textgen_model_2 = load_model('./model/pathfinder_wordtoken_model_200_epoch_withcuda_model.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [106]:
# select a seed text
#word_seed_text = lines[randint(0,len(lines))]
print('> using word tokenisation')
print('> seed text:')
print(textwrap.fill('%s' % (word_seed_text), 80) + '\n')

> using word tokenisation
> seed text:
of the mausoleum and attack the pcs as they enter to retrieve the magic candle
the four skeletons are dressed like fact these are the remains of the thugs who
activated the candle the skeletal champion wears the garb of a cleric of desna
identifiable with a successful dc knowledge local



In [0]:
# generate new text
temperature_table = [0, 1.0]

In [108]:
for temperature in temperature_table:
    generated = generate_seq_of_words(textgen_model_2, tokenizer, seq_length, 
                    word_seed_text, 100, temperature)
    print(">> generated text (temperature: {})".format(temperature))
    print(textwrap.fill('%s' % (generated), 80))
    print()

>> generated text (temperature: 0)
or knowledge religion check defeating the undead is easier if the pcs extinguish
the candle of the development section with the skeletons defeated the pcs can
deal with the candle of night with a successful dc knowledge arcana or knowledge
religion check the pcs learn this minor magic item cannot be extinguished save
by snuffing the flame with live flowing blood hazelindra adds that the pcs can
keep the candle as long as they do not tell the academy of her connection to
this situation the cemetery is half a mile west of the town and is accessible
via a



/gdrive/My Drive/Colab Notebooks/snaug/lib/data_common.py:143: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


>> generated text (temperature: 1.0)
or knowledge religion check defeating the undead is easier if the pcs extinguish
the candle of the development section with the skeletons defeated the pcs can
deal with the candle of night with a successful dc knowledge arcana or knowledge
religion check the pcs learn this minor magic item cannot be extinguished save
by snuffing the flame with live flowing blood hazelindra adds that the pcs can
keep the candle as long as they do not tell that about a long plum sized ruby
calling it the fire of versade savasha versade has decided to display it
publicly for the



In [0]:
#
# Word2vec pre-trained model
#

In [110]:
# load trained model
textgen_model_3 = load_model('./model/pathfinder_wordtoken_w2v_model_50_epoch_withcuda_model.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [111]:
# select a seed text
#word_seed_text = lines[randint(0,len(lines))]
print('> using word tokenisation and pre-trained model')
print('> seed text:')
print(textwrap.fill('%s' % (word_seed_text), 80) + '\n')

> using word tokenisation and pre-trained model
> seed text:
of the mausoleum and attack the pcs as they enter to retrieve the magic candle
the four skeletons are dressed like fact these are the remains of the thugs who
activated the candle the skeletal champion wears the garb of a cleric of desna
identifiable with a successful dc knowledge local



In [0]:
# generate new text
temperature_table = [0, 1.0]

In [113]:
for temperature in temperature_table:
    generated = generate_seq_of_words(textgen_model_3, tokenizer, seq_length, 
                    word_seed_text, 100, temperature)
    print(">> generated text (temperature: {})".format(temperature))
    print(textwrap.fill('%s' % (generated), 80))
    print()

>> generated text (temperature: 0)
or knowledge religion check defeating the undead is easier if the pcs extinguish
the candle of the development section with the skeletons defeated the pcs can
deal with the candle of night with a successful dc knowledge arcana or knowledge
religion check the pcs learn this minor magic item cannot be extinguished save
by snuffing the flame with live flowing blood in order for the pcs to put out
its flame and prevent more undead from rising from graves along their path back
to sandpoint they must douse the candle in blood from an open wound dealing at
least points



/gdrive/My Drive/Colab Notebooks/snaug/lib/data_common.py:143: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


>> generated text (temperature: 1.0)
or knowledge religion check defeating the undead is easier if the pcs extinguish
the candle of the development section with the skeletons defeated the pcs can
deal with the candle of night with a successful dc knowledge local or knowledge
religion check defeating the undead is easier if the pcs extinguish the candle
of the development section with the skeletons defeated the pcs can deal with the
candle of night with a successful dc knowledge arcana or knowledge religion
check the pcs learn this minor magic item cannot be extinguished save by
snuffing the flame with live flowing blood in

